In [1]:
import re 
import pickle 
import numpy as np 
import pandas as pd 
from nltk.stem import WordNetLemmatizer
# sklear n
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from nltk.corpus import stopwords

In [2]:
DATASET_COLUMS = ["sentiment", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
dataset = pd.read_csv('./../data/training.csv', encoding= DATASET_ENCODING, names = DATASET_COLUMS)
# Removing the unecessary columns 
dataset = dataset[['sentiment', 'text']]
# replacing the value to ease understanding 
dataset['sentiment'] = dataset['sentiment'].replace(4,1)
# storing the data in lists 
text ,sentiment = list(dataset['text']), list(dataset['sentiment'])

In [3]:
# grinning face
print("\U0001f600")
 
# grinning squinting face
print("\U0001F609")
 
# rolling on the floor laughing
print("\U0001F923")

😀
😉
🤣


In [4]:
# defining dictionary containing all emojis with their meaning, 
emojis = {
    ':)':'smile', ':-)':'smile',';d':'wink', ':-E':'vampire',':(':'sad',':-(':'sad', ':-<':'sad',':P':'raspberry',':O':'surprised',
    ':-@':'shocked',':@':'shocked',':-$':'confused',':\\':'annoyed', ':#':'mute', ':X':'mute', ':^)':'smile', ':-&':'confused','$_$':'greedy',
    '@@':'eyeroll',':-!':'confused',':-D':'smile',':-0':'yell','O.o':'confused',
    '<(-_-)>':'robot','d[-_-]b':'dj',":'-)":'sadsmile',';)':'wink',';-)':'wink','O:-)':'angel', 'O*-)':'angel','(:-D':'gossip', '=^.^=':'cat'}

In [5]:
emojis

{':)': 'smile',
 ':-)': 'smile',
 ';d': 'wink',
 ':-E': 'vampire',
 ':(': 'sad',
 ':-(': 'sad',
 ':-<': 'sad',
 ':P': 'raspberry',
 ':O': 'surprised',
 ':-@': 'shocked',
 ':@': 'shocked',
 ':-$': 'confused',
 ':\\': 'annoyed',
 ':#': 'mute',
 ':X': 'mute',
 ':^)': 'smile',
 ':-&': 'confused',
 '$_$': 'greedy',
 '@@': 'eyeroll',
 ':-!': 'confused',
 ':-D': 'smile',
 ':-0': 'yell',
 'O.o': 'confused',
 '<(-_-)>': 'robot',
 'd[-_-]b': 'dj',
 ":'-)": 'sadsmile',
 ';)': 'wink',
 ';-)': 'wink',
 'O:-)': 'angel',
 'O*-)': 'angel',
 '(:-D': 'gossip',
 '=^.^=': 'cat'}

In [6]:
import nltk 
nltk.download('omw-1.4')

[nltk_data] Error loading omw-1.4: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

In [7]:
lemmatizer = WordNetLemmatizer()
# grouping toghther the inflected form ('better.>good)
stopwords = set(stopwords.words('english'))

In [8]:
def preprocess(textdata):
    processed_texts = []
    # Defining regex patterns 
    url_pattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    user_pattern = '@[^\s]+'
    alpha_pattern = "[^a-zA-Z0-9]"
    sequence_pattern = r"(.)\1\1+"
    seq_replace_pattern = r"\1\1"

    for tweet in textdata:
        tweet = tweet.lower()
        # replace all urls with 'url'
        tweet = re.sub(url_pattern, 'URL', tweet)
        # Replace all emojis 
        for emoji in emojis.keys():
            tweet = tweet.replace(emoji, 'EMOJI'+emojis[emoji])
        # replace @username to user 
        tweet = re.sub(user_pattern, 'USER', tweet)
        # replace all non alphabets
        tweet = re.sub(alpha_pattern, " ", tweet)
        # replace 3 or more consecutive letters by 2 letter. 
        tweet = re.sub(sequence_pattern, seq_replace_pattern,tweet)

        preprocessed_words = []
        for word in tweet.split():
            if len(word)>1 and word not in stopwords:
                # lemmatize the word 
                word = lemmatizer.lemmatize(word)
                preprocessed_words.append(word)
        processed_texts.append(' '.join(preprocessed_words))
    return processed_texts

In [9]:
preprocessedtext = preprocess(text)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(preprocessedtext, sentiment, test_size= 0.3, random_state=0)

In [17]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=500000)
vectorizer.fit(X_train)

TfidfVectorizer(max_features=500000, ngram_range=(1, 2))

In [18]:
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [19]:
def model_evaluate(model):
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

In [20]:
while True:
    for i in X_train:
        print(i)
        break
    break

  (0, 459540)	0.12350740062498185
  (0, 457025)	0.2728345560451567
  (0, 456516)	0.07453853684622454
  (0, 404190)	0.2728345560451567
  (0, 403841)	0.12759108212772935
  (0, 396536)	0.16829060432345327
  (0, 387245)	0.29064040848423905
  (0, 387227)	0.1911713616790475
  (0, 385613)	0.28459480779181556
  (0, 385513)	0.14805801822751993
  (0, 328798)	0.25681828368202336
  (0, 327784)	0.23842042399047453
  (0, 265952)	0.13444360003802303
  (0, 228206)	0.17225108305559025
  (0, 223347)	0.1787780580033966
  (0, 161045)	0.2675532165835685
  (0, 159838)	0.08170817412921148
  (0, 121558)	0.11410796065007721
  (0, 57144)	0.2675532165835685
  (0, 57143)	0.21674145330092714
  (0, 20834)	0.21383153141569364
  (0, 8096)	0.27002841149526857
  (0, 8067)	0.17546550659516488


In [21]:
BNBModel = BernoulliNB(alpha = 2)
BNBModel.fit(X_train, y_train)
model_evaluate(BNBModel)

              precision    recall  f1-score   support

           0       0.79      0.77      0.78    239877
           1       0.77      0.80      0.78    240123

    accuracy                           0.78    480000
   macro avg       0.78      0.78      0.78    480000
weighted avg       0.78      0.78      0.78    480000



In [22]:
SVCmodel = LinearSVC()
SVCmodel.fit(X_train, y_train)
model_evaluate(SVCmodel)

              precision    recall  f1-score   support

           0       0.79      0.77      0.78    239877
           1       0.77      0.80      0.78    240123

    accuracy                           0.78    480000
   macro avg       0.78      0.78      0.78    480000
weighted avg       0.78      0.78      0.78    480000



In [23]:
LRmodel = LogisticRegression(C=2, max_iter = 1000 ,n_jobs = -1)
LRmodel.fit(X_train, y_train)
model_evaluate(LRmodel)

              precision    recall  f1-score   support

           0       0.81      0.78      0.79    239877
           1       0.79      0.81      0.80    240123

    accuracy                           0.79    480000
   macro avg       0.80      0.79      0.79    480000
weighted avg       0.80      0.79      0.79    480000



In [24]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
model_evaluate(rf)

In [22]:
# Pipeline 
from sklearn.pipeline import Pipeline 
X_train, X_test, y_train, y_test = train_test_split(preprocessedtext , sentiment, test_size = 0.5, random_state = 0)
pipe = Pipeline([('vectorizer',vectorizer), ('bnb', BNBModel)])
pipe.fit(X_train, y_train)
model_evaluate(pipe)                

              precision    recall  f1-score   support

           0       0.79      0.76      0.77    399751
           1       0.77      0.79      0.78    400249

    accuracy                           0.78    800000
   macro avg       0.78      0.78      0.78    800000
weighted avg       0.78      0.78      0.78    800000



In [24]:
with open('pipeline.pickle', 'wb') as f:
    pickle.dump(pipe, f)

with open('pipeline.pickle', 'rb') as f:
    loaded_pipe = pickle.load(f)
model_evaluate(loaded_pipe)

              precision    recall  f1-score   support

           0       0.79      0.76      0.77    399751
           1       0.77      0.79      0.78    400249

    accuracy                           0.78    800000
   macro avg       0.78      0.78      0.78    800000
weighted avg       0.78      0.78      0.78    800000



In [25]:
def predict(model, text):
    # predict the sentiment
    preprocessed_text = preprocess(text)
    predictions = model.predict(preprocessed_text)
    pred_to_label = {0:'Negative', 1:'Positive'}
    # Make a list of text with sentimet 
    data = []
    for t, pred in zip(text, predictions):
        data.append((t, pred, pred_to_label[pred]))
    return data 
if __name__ == "__main__":
    # Text to classify shold be in a list 
    text = ["I hate twitter", 
           "May the force be with you.",
           "Mr. stark, I don't feel so good. "]
    predictions = predict(loaded_pipe, text)
    print(predictions)

[('I hate twitter', 0, 'Negative'), ('May the force be with you.', 1, 'Positive'), ("Mr. stark, I don't feel so good. ", 1, 'Positive')]
